In [1]:
import os
import json
from collections import deque
from typing import Dict, List, Optional, Any, Union
from pydantic import BaseModel

from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain import OpenAI, SerpAPIWrapper, LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI


from langchain.vectorstores import Chroma
from langchain.docstore import InMemoryDocstore

from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import chromadb
import re


# Get human objective
# refine it via human in the loop and via adding context (short/long term memory search) -> the output is a json dict of the design
# build the modules and test benches usiiing short and long term memory

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [3]:
llm = ChatOpenAI(model='gpt-4', temperature=0)
bigllm = ChatOpenAI(model='gpt-4-1106-preview', temperature=0)

In [4]:
from FPGA_AGI.tools import *
from FPGA_AGI.agents import *
from FPGA_AGI.utils import *

## FPGA-AGI

In [5]:
from pydantic import BaseModel
import warnings

class FPGA_AGI(BaseModel):
    verbose: bool = False
    solution_num: int = 0
    requirement_agent_executor: RequirementAgentExecutor = None
    module_agent_executor: ModuleAgentExecutor = None
    hdl_agent_executor: HdlAgentExecutor = None
    requirement: str = ''
    project_details: ProjectDetails = None
    module_list: List = []
    codes: List = []
    module_list_str: List = []

    def __init__(self, **data):
        super().__init__(**data)
        if (self.requirement_agent_executor is None) or (self.module_agent_executor is None) or (self.hdl_agent_executor is None):
            warnings.warn("RequirementAgentExecutor, ModuleAgentExecutor and HdlAgentExecutor are not set. Please use the from_llm class method to properly initialize it.", UserWarning)

    @classmethod
    def from_llm(cls, llm, verbose=False):
        tools = [
            web_search_tool,
            document_search_tool,
            #human_input_tool
        ]
        requirement_agent_executor = RequirementAgentExecutor.from_llm_and_tools(llm=llm, tools=tools, verbose=verbose)
        tools = [
            web_search_tool,
            document_search_tool,
        ]
        module_agent_executor = ModuleAgentExecutor.from_llm_and_tools(llm=llm, tools=tools, verbose=verbose)
        tools = [
            #web_search_tool,
            document_search_tool,
        ]
        hdl_agent_executor = HdlAgentExecutor.from_llm_and_tools(llm=llm, tools=tools, verbose=verbose)
        return cls(verbose=verbose, requirement_agent_executor=requirement_agent_executor, module_agent_executor=module_agent_executor, hdl_agent_executor=hdl_agent_executor)

    def _run(self, objective, action_type):
        if action_type == 'full':
            self.requirement = self.requirement_agent_executor.run(objective)
            self.project_details = extract_project_details(self.requirement)
        elif action_type == 'module':
            assert type(objective) == ProjectDetails, "objective needs to be of ProjectDetails type for module level design"
            self.project_details = objective
        if action_type == 'hdl':
            try:
                self.module_list = json.loads(objective)
            except json.JSONDecodeError as e:
                raise FormatError
        else:
            self.module_list = self.module_agent_executor.run(Goals=self.project_details.goals, Requirements=self.project_details.requirements, Constraints=self.project_details.constraints)
            self.module_list = extract_json_from_string(self.module_list)
        self.codes = []
        self.module_list_str = [str(item) for item in self.module_list]
        #current_modules_verified = json.loads(current_modules_verified)
        for module in self.module_list:
            code = self.hdl_agent_executor.run(
                Goals=self.project_details.goals,
                Requirements=self.project_details.requirements,
                Constraints=self.project_details.constraints,
                module_list='\n'.join(self.module_list_str),
                codes='\n'.join(self.codes),
                module=str(module)
                )
            self.codes.append(code)
        save_solution(self.codes, solution_num=self.solution_num)
        self.solution_num += 1

    def __call__(self, objective: Any, action_type: str = 'full'):
        if (self.requirement_agent_executor is None) or (self.module_agent_executor is None) or (self.hdl_agent_executor is None):
            raise Exception("RequirementAgentExecutor, ModuleAgentExecutor and HdlAgentExecutor are not set. Please use the from_llm class method to properly initialize it.")
        if not (action_type in ['full', 'module', 'hdl']):
            raise ValueError("Invalid action type specified.")
        self._run(objective, action_type)

In [6]:
fpga_agi_instance = FPGA_AGI.from_llm(llm=bigllm, verbose=True)

fpga_agi_instance(objective="design a very simple 32 bit risc-v cpu in verilog. The cpu will be a softcore solution for zync ultrascale devices and it must connect to the PS side of the FPGA via AXI-stream for the data and AXI-lite for the control. As a note, I am going to use pynq to connect to this device so these axi interfaces must be pynq compatible as much as possible." ,action_type='hdl')
#fpga_agi_instance(objective="design a very simple 32 bit risc-v cpu in verilog. The cpu will be a softcore solution for zync ultrascale devices and it must connect to the PS side of the FPGA via AXI-stream for the data and AXI-lite for the control. As a note, I am going to use pynq to connect to this device so these axi interfaces must be pynq compatible as much as possible." ,action_type='module')

FormatError: Input should be of the following form:
{
    "Module_Name": "name of the module",
    "ports": ["specific inputs and outputs, including bit width"],
    "description": "detailed description of the module function",
    "connections": ["specific other modules it must connect to"],
    "hdl_language": "hdl language to be used"
}